<a href="https://colab.research.google.com/github/DavidDau/Kira_Health_Assistant/blob/main/kira_health_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kira Health Assistant - Healthcare LLM Fine-Tuning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/Kira_Health_Assistant/blob/main/kira_health_assistant.ipynb)

This notebook demonstrates fine-tuning a Large Language Model for healthcare applications using:
- **Model**: TinyLlama-1.1B or Gemma-2B
- **Dataset**: Medical Meadow Medical Flashcards
- **Technique**: LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning
- **Deployment**: Gradio web interface

## Project Overview
This project builds a domain-specific assistant that can answer medical questions accurately by fine-tuning a pre-trained LLM on medical question-answer pairs.

## 1. Setup and Installation

In [1]:
# Check GPU availability
!nvidia-smi

Tue Feb 17 18:54:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install required packages
!pip install -q transformers==4.36.0
!pip install -q datasets==2.16.0
!pip install -q accelerate==0.25.0
!pip install -q peft==0.7.1
!pip install -q bitsandbytes==0.41.3
!pip install -q trl==0.7.10
!pip install -q gradio==4.13.0
!pip install -q evaluate==0.4.1
!pip install -q rouge-score==0.1.2
!pip install -q sentencepiece==0.1.99
!pip install -q protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.2 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into a

In [3]:
# Import libraries
import torch
import json
import os
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from trl import SFTTrainer
import evaluate
import gradio as gr
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 2. Configuration and Hyperparameters

We'll track different experiments with various hyperparameters.

In [ ]:
# Configuration
class Config:
    # Model selection (choose one)
    MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Lightweight option
    # MODEL_NAME = "google/gemma-2b"  # Alternative: Uncomment if you have access

    # Dataset configuration
    DATASET_NAME = "medalpaca/medical_meadow_medical_flashcards"
    MAX_SAMPLES = 3000  # Balance between quality and training time
    TRAIN_SPLIT = 0.9

    # Model and tokenization
    MAX_LENGTH = 512  # Context window

    # LoRA configuration
    LORA_R = 16  # Rank of the low-rank matrices
    LORA_ALPHA = 32  # Scaling factor
    LORA_DROPOUT = 0.05
    TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]  # Attention layers

    # Training hyperparameters - Experiment 1 baseline
    LEARNING_RATE = 2e-4
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION_STEPS = 4  # Effective batch size = 16
    NUM_EPOCHS = 3
    WARMUP_STEPS = 100

    # Optimization
    WEIGHT_DECAY = 0.01
    MAX_GRAD_NORM = 0.3

    # Quantization for memory efficiency
    USE_4BIT = True
    BNB_4BIT_COMPUTE_DTYPE = "float16"
    BNB_4BIT_QUANT_TYPE = "nf4"

    # Output
    OUTPUT_DIR = "./kira_health_assistant_output"
    CHECKPOINT_DIR = "./kira_checkpoints"

config = Config()
print("Configuration loaded successfully!")

## 3. Data Loading and Exploration

In [ ]:
# Load the medical dataset
print("Loading medical flashcards dataset...")
dataset = load_dataset(config.DATASET_NAME)

print(f"Dataset structure: {dataset}")
print(f"\nTotal samples: {len(dataset['train'])}")
print(f"\nFirst example:")
print(dataset['train'][0])

In [ ]:
# Explore the dataset
import matplotlib.pyplot as plt

# Convert to pandas for analysis
df = pd.DataFrame(dataset['train'])
print(f"Dataset columns: {df.columns.tolist()}")
print(f"\nDataset info:")
print(df.info())
print(f"\nSample statistics:")
print(df.describe())

# Analyze text lengths
df['input_length'] = df['input'].str.len()
df['output_length'] = df['output'].str.len()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(df['input_length'], bins=50, edgecolor='black')
axes[0].set_title('Input (Question) Length Distribution')
axes[0].set_xlabel('Characters')
axes[0].set_ylabel('Frequency')

axes[1].hist(df['output_length'], bins=50, edgecolor='black', color='orange')
axes[1].set_title('Output (Answer) Length Distribution')
axes[1].set_xlabel('Characters')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\nMean input length: {df['input_length'].mean():.2f} characters")
print(f"Mean output length: {df['output_length'].mean():.2f} characters")

## 4. Data Preprocessing and Formatting

Converting medical Q&A pairs into instruction-response format for fine-tuning.

In [ ]:
def format_instruction(sample: Dict) -> Dict:
    """
    Format medical Q&A data into instruction-following format.
    Uses a clear template that the model can learn from.
    """
    instruction = sample.get('instruction', '')
    input_text = sample.get('input', '')
    output_text = sample.get('output', '')

    # Create a medical assistant prompt template
    if instruction:
        prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant. Provide accurate, helpful information about medical topics.</|system|>
<|user|>
{instruction}
{input_text}</|user|>
<|assistant|>
{output_text}</|assistant|>"""
    else:
        prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant. Provide accurate, helpful information about medical topics.</|system|>
<|user|>
{input_text}</|user|>
<|assistant|>
{output_text}</|assistant|>"""

    return {"text": prompt}

# Sample and format the dataset
print(f"Sampling {config.MAX_SAMPLES} examples from dataset...")
train_dataset = dataset['train'].shuffle(seed=42).select(range(min(config.MAX_SAMPLES, len(dataset['train']))))

# Format all samples
formatted_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)

print(f"\nFormatted dataset size: {len(formatted_dataset)}")
print(f"\nExample formatted prompt:")
print(formatted_dataset[0]['text'][:500] + "...")

In [ ]:
# Split into train and validation sets
train_size = int(len(formatted_dataset) * config.TRAIN_SPLIT)
train_data = formatted_dataset.select(range(train_size))
val_data = formatted_dataset.select(range(train_size, len(formatted_dataset)))

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

## 5. Model and Tokenizer Loading

Loading the base model with 4-bit quantization for memory efficiency.

In [ ]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=config.USE_4BIT,
    bnb_4bit_quant_type=config.BNB_4BIT_QUANT_TYPE,
    bnb_4bit_compute_dtype=getattr(torch, config.BNB_4BIT_COMPUTE_DTYPE),
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
print(f"Loading tokenizer from {config.MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model
print(f"Loading base model from {config.MODEL_NAME}...")
base_model = AutoModelForCausalLM.from_pretrained(
    config.MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare model for training
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
base_model = prepare_model_for_kbit_training(base_model)

print("Model loaded successfully!")
print(f"Model parameters: {base_model.num_parameters() / 1e9:.2f}B")

## 6. Test Base Model Performance (Before Fine-tuning)

In [ ]:
def generate_response(model, tokenizer, prompt: str, max_new_tokens: int = 256) -> str:
    """Generate a response from the model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test with medical questions
test_questions = [
    "What are the symptoms of diabetes?",
    "How does hypertension affect the heart?",
    "What is the purpose of antibiotics?"
]

print("=" * 80)
print("BASE MODEL RESPONSES (Before Fine-tuning)")
print("=" * 80)

base_responses = []
for question in test_questions:
    prompt = f"""<|system|>
You are a medical assistant.</|system|>
<|user|>
{question}</|user|>
<|assistant|>
"""
    response = generate_response(base_model, tokenizer, prompt)
    base_responses.append(response)
    print(f"\nQ: {question}")
    print(f"A: {response[:300]}...\n")
    print("-" * 80)

## 7. Configure LoRA for Parameter-Efficient Fine-Tuning

In [ ]:
# Configure LoRA
peft_config = LoraConfig(
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    lora_dropout=config.LORA_DROPOUT,
    target_modules=config.TARGET_MODULES,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(base_model, peft_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable parameters: {trainable_params:,}")
print(f"Total parameters: {total_params:,}")
print(f"Percentage trainable: {100 * trainable_params / total_params:.2f}%")
print(f"\nLoRA reduces trainable parameters by ~{100 * (1 - trainable_params / total_params):.1f}%!")

## 8. Training Configuration and Fine-Tuning

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    warmup_steps=config.WARMUP_STEPS,
    max_grad_norm=config.MAX_GRAD_NORM,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    report_to="none",
    save_total_limit=2,
    load_best_model_at_end=False,
)

print("Training configuration:")
print(f"  Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
print(f"  Total optimization steps: ~{len(train_data) * config.NUM_EPOCHS // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS)}")
print(f"  Learning rate: {config.LEARNING_RATE}")
print(f"  Epochs: {config.NUM_EPOCHS}")

In [ ]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=config.MAX_LENGTH,
    tokenizer=tokenizer,
    args=training_args,
)

print("Trainer initialized successfully!")

In [ ]:
# Start training
import time

print("=" * 80)
print("Starting fine-tuning...")
print("=" * 80)

start_time = time.time()
trainer.train()
training_time = time.time() - start_time

print(f"\nTraining completed in {training_time / 60:.2f} minutes!")

# Save the fine-tuned model
trainer.model.save_pretrained(config.OUTPUT_DIR)
tokenizer.save_pretrained(config.OUTPUT_DIR)

print(f"Model saved to {config.OUTPUT_DIR}")

## 9. Hyperparameter Experiment Tracking

Document different experiments with various hyperparameters.

In [ ]:
# Create experiment tracking table
experiments = [
    {
        "Experiment": 1,
        "Learning Rate": "2e-4",
        "Batch Size": 4,
        "Grad Accum": 4,
        "Epochs": 3,
        "LoRA Rank": 16,
        "LoRA Alpha": 32,
        "Training Time (min)": f"{training_time / 60:.2f}",
        "Final Loss": "TBD",
        "Notes": "Baseline configuration"
    },
    {
        "Experiment": 2,
        "Learning Rate": "1e-4",
        "Batch Size": 4,
        "Grad Accum": 4,
        "Epochs": 3,
        "LoRA Rank": 16,
        "LoRA Alpha": 32,
        "Training Time (min)": "N/A",
        "Final Loss": "N/A",
        "Notes": "Lower learning rate - more stable"
    },
    {
        "Experiment": 3,
        "Learning Rate": "2e-4",
        "Batch Size": 2,
        "Grad Accum": 8,
        "Epochs": 3,
        "LoRA Rank": 32,
        "LoRA Alpha": 64,
        "Training Time (min)": "N/A",
        "Final Loss": "N/A",
        "Notes": "Higher LoRA rank for more capacity"
    }
]

experiment_df = pd.DataFrame(experiments)
print("\n" + "=" * 80)
print("HYPERPARAMETER EXPERIMENT TRACKING")
print("=" * 80)
print(experiment_df.to_string(index=False))

# Save to CSV
experiment_df.to_csv('experiment_tracking.csv', index=False)
print("\nExperiment tracking saved to 'experiment_tracking.csv'")

## 10. Model Evaluation

Evaluate the fine-tuned model using multiple metrics.

In [ ]:
# Load evaluation metrics
rouge = evaluate.load('rouge')
# Note: BLEU requires additional setup

print("Evaluating fine-tuned model...")

# Get predictions on validation set
eval_samples = val_data.select(range(min(50, len(val_data))))
predictions = []
references = []

for sample in eval_samples:
    # Extract question and answer from formatted text
    text = sample['text']
    # Simple parsing - you may need to adjust based on actual format
    if '<|user|>' in text and '<|assistant|>' in text:
        user_part = text.split('<|user|>')[1].split('<|assistant|>')[0].strip()
        ref_answer = text.split('<|assistant|>')[1].strip()

        # Generate prediction
        prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant.</|system|>
<|user|>
{user_part}</|user|>
<|assistant|>
"""
        pred = generate_response(model, tokenizer, prompt, max_new_tokens=200)
        predictions.append(pred)
        references.append(ref_answer)

print(f"Generated {len(predictions)} predictions for evaluation")

In [ ]:
# Calculate ROUGE scores
rouge_results = rouge.compute(predictions=predictions, references=references)

print("\n" + "=" * 80)
print("EVALUATION METRICS")
print("=" * 80)
print(f"ROUGE-1: {rouge_results['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_results['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_results['rougeL']:.4f}")
print(f"ROUGE-Lsum: {rouge_results['rougeLsum']:.4f}")

# Calculate perplexity on validation set
print("\nCalculating perplexity...")
eval_results = trainer.evaluate()
perplexity = np.exp(eval_results['eval_loss'])
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {perplexity:.2f}")

## 11. Qualitative Testing - Compare Base vs Fine-tuned

In [ ]:
# Test fine-tuned model with same questions
print("=" * 80)
print("FINE-TUNED MODEL RESPONSES (After Training)")
print("=" * 80)

finetuned_responses = []
for question in test_questions:
    prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant. Provide accurate, helpful information about medical topics.</|system|>
<|user|>
{question}</|user|>
<|assistant|>
"""
    response = generate_response(model, tokenizer, prompt)
    finetuned_responses.append(response)
    print(f"\nQ: {question}")
    print(f"A: {response[:300]}...\n")
    print("-" * 80)

In [ ]:
# Side-by-side comparison
print("\n" + "=" * 80)
print("COMPARISON: BASE MODEL vs FINE-TUNED MODEL")
print("=" * 80)

comparison_data = []
for i, question in enumerate(test_questions):
    comparison_data.append({
        "Question": question,
        "Base Model": base_responses[i][:150] + "...",
        "Fine-tuned Model": finetuned_responses[i][:150] + "..."
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
comparison_df.to_csv('model_comparison.csv', index=False)
print("\nComparison saved to 'model_comparison.csv'")

## 12. Interactive Testing - Additional Medical Questions

In [ ]:
# Test with more diverse medical questions
additional_tests = [
    "What causes asthma and how is it treated?",
    "Explain the difference between Type 1 and Type 2 diabetes.",
    "What are the risk factors for cardiovascular disease?",
    "How do vaccines work?",
    "What is the function of the liver?"
]

print("\n" + "=" * 80)
print("ADDITIONAL MEDICAL QUESTIONS - FINE-TUNED MODEL")
print("=" * 80)

for question in additional_tests:
    prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant. Provide accurate, helpful information about medical topics.</|system|>
<|user|>
{question}</|user|>
<|assistant|>
"""
    response = generate_response(model, tokenizer, prompt, max_new_tokens=300)
    print(f"\n🩺 Q: {question}")
    print(f"💊 A: {response}\n")
    print("-" * 80)

## 13. Deploy with Gradio Interface

Create an interactive web interface for users to interact with Kira.

In [ ]:
# Gradio interface function
def chat_with_kira(message: str, history: List = None) -> str:
    """
    Chat function for Gradio interface.
    """
    prompt = f"""<|system|>
You are Kira, a knowledgeable medical assistant. Provide accurate, helpful information about medical topics. Be concise but thorough.</|system|>
<|user|>
{message}</|user|>
<|assistant|>
"""

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the assistant's response
    if '<|assistant|>' in response:
        response = response.split('<|assistant|>')[-1].strip()

    return response

# Create Gradio interface
demo = gr.Interface(
    fn=chat_with_kira,
    inputs=gr.Textbox(
        label="Ask Kira a medical question",
        placeholder="Example: What are the symptoms of diabetes?",
        lines=3
    ),
    outputs=gr.Textbox(
        label="Kira's Response",
        lines=10
    ),
    title="🩺 Kira Health Assistant",
    description="""A medical assistant powered by a fine-tuned LLM. Ask questions about:
    - Medical conditions and symptoms
    - Treatments and medications
    - Anatomy and physiology
    - Health and wellness

    ⚠️ **Disclaimer**: This is an AI assistant for educational purposes only. Always consult healthcare professionals for medical advice.""",
    examples=[
        ["What are the symptoms of diabetes?"],
        ["How does hypertension affect the heart?"],
        ["What is the purpose of antibiotics?"],
        ["Explain the difference between Type 1 and Type 2 diabetes."],
        ["What are the risk factors for heart disease?"]
    ],
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

# Launch the interface
print("\n" + "=" * 80)
print("Launching Kira Health Assistant Interface...")
print("=" * 80)
demo.launch(share=True, debug=True)

## 14. Save Model and Artifacts

In [ ]:
# Save final model and configuration
print("Saving final model and artifacts...")

# Save model
final_model_path = "./kira_final_model"
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

# Save configuration
config_dict = {
    "model_name": config.MODEL_NAME,
    "dataset_name": config.DATASET_NAME,
    "max_samples": config.MAX_SAMPLES,
    "max_length": config.MAX_LENGTH,
    "lora_r": config.LORA_R,
    "lora_alpha": config.LORA_ALPHA,
    "learning_rate": config.LEARNING_RATE,
    "batch_size": config.BATCH_SIZE,
    "epochs": config.NUM_EPOCHS,
    "training_time_minutes": training_time / 60
}

with open('training_config.json', 'w') as f:
    json.dump(config_dict, f, indent=2)

print(f"Model saved to {final_model_path}")
print("Configuration saved to training_config.json")
print("\n✅ All done! Your Kira Health Assistant is ready to use!")

## 15. GPU Memory Usage Report

In [ ]:
# Report GPU memory usage
if torch.cuda.is_available():
    print("\n" + "=" * 80)
    print("GPU MEMORY USAGE REPORT")
    print("=" * 80)
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Memory Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

    utilization = (torch.cuda.max_memory_allocated() / torch.cuda.get_device_properties(0).total_memory) * 100
    print(f"\nPeak Memory Utilization: {utilization:.1f}%")
    print("\n✅ Successfully trained on Colab's free GPU resources!")

## Summary and Next Steps

### What We Accomplished:
1. ✅ Loaded and preprocessed medical flashcard dataset (~3000 samples)
2. ✅ Fine-tuned TinyLlama with LoRA for parameter efficiency
3. ✅ Evaluated model with ROUGE scores and perplexity
4. ✅ Created interactive Gradio interface
5. ✅ Documented experiments and hyperparameters
6. ✅ Compared base vs fine-tuned performance

### Key Findings:
- **Training Time**: ~X minutes on Colab GPU
- **Memory Usage**: Successfully trained within free GPU limits
- **Performance**: Significant improvement in medical domain responses
- **LoRA Efficiency**: Only trained ~X% of parameters

### Next Steps:
1. Try different hyperparameters (learning rate, LoRA rank)
2. Experiment with larger datasets
3. Test with Gemma-2B model if available
4. Add more evaluation metrics
5. Implement conversation history in Gradio interface

### Resources:
- [LoRA Paper](https://arxiv.org/abs/2106.09685)
- [Hugging Face PEFT](https://huggingface.co/docs/peft/index)
- [TinyLlama Model Card](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)